# Solving TSP with Quantum
In this notebook, we use D-Wave's Advantage System 1.1 quantum annealer to solve the Traveling Salesperson Problem.


*you will need a D-Wave Leap account to run this notebook without errors*


In [1]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from dwave.plugins.qiskit import DWaveMinimumEigensolver
from qiskit.optimization.applications.ising import tsp
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
import numpy as np

## Define Functions
We're already operating at a high level of abstraction because we're using Qiskit to solve the TSP. Defining these functions adds another layer of abstraction to keep this notebook tidy and readable.

### Function 1: Generate n Cities
This effectively defines n random points in 2D space, then converts these points into a TSP that can be solved by our D-Wave quantum computer.

In [2]:
def get_op(ncity):
    #Seed is set to 123 for reproducibility
    print(f'Generating {ncity} cities...')
    ncities_tsp = tsp.random_tsp(ncity, seed=123)
    print(f'Converting TSP with {ncity} cities into an operator...')
    operator, offset = tsp.get_operator(ncities_tsp)
    print(f'Returning operator with {ncity} cities, requiring {operator.num_qubits} qubits with a total of {np.power(2, operator.num_qubits)} states.')
    return operator

### Function 2: Send Operator to QC
This function creates an Eigenvalue solver to calculate the solution to our TSP. This quantum Eigensolver is then run on real quantum hardware by D-Wave.

In [3]:
def qpu_run(op, nread):
    dwave_mes = DWaveMinimumEigensolver(num_reads=nread)
    result = dwave_mes.compute_minimum_eigenvalue(op)
    return result

### Function 3: Check Feasibility
The "best" solution our QC generates won't necessarily be a valid solution. This function produces a warning should it detect the solution determined by the QC is not a valid one.

In [4]:
def is_feasible(tspx):
    tspf = tsp.tsp_feasible(tspx)
    if tspf == True:
        print('TSP is feasible.')
    elif tspf == False:
        print('ERROR: TSP is not feasible.')
    else:
        print('Something weird is happening, TSP does not seem to be feasible or not-feasible.')

### Function 4: Process Result from QC
In order to find the solution to our problem, we need to aggregate our readings from each shot. Basically we're just looking for the lowest energy readings over all of the shots, which we translate to a sequence of cities. 

In [5]:
def get_solution(result):
    tspx = sample_most_likely(result.eigenstate)
    is_feasible(tspx)
    tsp_solution = tsp.get_tsp_solution(tspx)
    print(tspx)
    return tsp_solution

### Function 5: Attempt Solving as a Classic Computer function
To compare the processing advantage of our quantum computer to a classical computer, we give the classical computer running this notebook (hosted on an AWS t3.medium instance) the same problem we gave the quantum computer. 

In [6]:
def try_solve_classic():
    print('WARNING: This function will almost certainly break!!!!!!!!!!!!!!')
    classic_result = NumPyMinimumEigensolver().compute_minimum_eigenvalue(operator)
    return classic_result

### Function 6: Solve a TSP for n Cities
Now, to put all the pieces of this together. We generate random points on a 2D plane, turn that data into an equation to find the shortest total path through each of the points, solve that equation with a quantum computer, and convert the amplitude measurements from the QC back into results that we can easily interpret.

In [7]:
def solve_ncity(ncity, nread):
    print(f'-> Cities: {ncity} \n-> Shots: {nread}')
    print('[1/3] Getting operator...')
    op = get_op(ncity)
    print('[2/3] Running on QPU...')
    result = qpu_run(op, nread)
    print('[3/3] Getting solution...')
    solution = get_solution(result)
    print(f'--> Best solution to TSP: {solution}')

## Finally, the results!
We generate 7 cities and take 10k shots to find our solution out of a pool of 562,949,953,421,312 possible solutions. All of this is done in just a few hundred milliseconds.

In [8]:
solve_ncity(7, 10000)

-> Cities: 7 
-> Shots: 10000
[1/3] Getting operator...
Generating 7 cities...
Converting TSP with 7 cities into an operator...
Returning operator with 7 cities, requiring 49 qubits with a total of 562949953421312 states.
[2/3] Running on QPU...
[3/3] Getting solution...
TSP is feasible.
[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 0]
--> Best solution to TSP: [0, 6, 1, 5, 4, 2, 3]


#### Notebook by Brian Lechthaler

GPL v3 | 2020-12-11 | please do not redistribute without attribution

* https://twitter.com/brianlechthaler
* https://github.com/brianlechthaler
* This notebook belongs to this repository: https://github.com/brianlechthaler/QuantumSolutions